In [1]:
! pip install ollama

In [2]:
! pip install statsmodels

In [2]:
import pandas as pd 
import numpy as np 
import json 
import time
import mlflow
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
import json 
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import sys
sys.path.insert(0,r"C:Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/src")
from data.data_cleaning  import CleanData
from data.data_preprocessing import DataPreprocessing
from models.model import BiLSTMForecast
from models.training import Trainer 
from pipelines.forecasting_pipeline import ForecastingPipeline
import pandas as pd 
import re
import os 
from meta_HPO.meta_knowledge_building import MetaKnowledgeBuilder
from meta_HPO.llm_prompt import prompt_template
from meta_HPO.LLM_Load import load_llm


# 1 .Meta-Knowledge Construction

In [4]:
#-----------test forecasting Pipelien----------
# load Clean train/ test data
Target_Cols = ['T (degC)', 'rh (%)', 'p (mbar)', 'wv (m/s)']


clean_train = pd.read_csv('C:/Users/user.IBRAHIM-IK-SZHE/pape_project/hyperopt-llm-project/data/clean_data/clean_train.csv',
parse_dates=["datetime"],
index_col="datetime"
)

# MLflow experiment name (must already exist)
EXPERIMENT_NAME = "Bayesian_Optimization"
meta_builder= MetaKnowledgeBuilder(clean_train=clean_train,experiment_name=EXPERIMENT_NAME,target_cols=Target_Cols)

top_k = 3

best_trials = meta_builder.create_meta_trials(top_k=top_k)

print("✅ Best historical trials:")
for t in best_trials:
    print(t)

meta_features = meta_builder.meta_features_creation()

print("✅ Dataset regime:")
print(meta_features["dataset_regime"])

print("\n✅ Aggregated meta-features:")
print(meta_features["aggregated_meta"])


c:\Users\user.IBRAHIM-IK-SZHE\anaconda3\envs\HPO311\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


✅ Best historical trials:
{'run_id': 'trial_1', 'test_rmse': 1.1170297250018493, 'params': {'optimizer': 'adamax', 'epochs': '17', 'num_layers': '1', 'batch_size': '32', 'hidden_size': '126', 'lr': '0.0005182120604022333', 'horizon': '4', 'dropout': '0.0', 'input_dim': '4', 'model_name': 'Bi-LSTM', 'seed': '42', 'lag': '58'}}
{'run_id': 'trial_2', 'test_rmse': 1.9278644352798011, 'params': {'optimizer': 'adamw', 'epochs': '6', 'num_layers': '1', 'batch_size': '32', 'hidden_size': '81', 'lr': '0.0003623165345530821', 'horizon': '4', 'dropout': '0.0', 'input_dim': '4', 'model_name': 'Bi-LSTM', 'seed': '42', 'lag': '38'}}
{'run_id': 'trial_3', 'test_rmse': 2.7698660920690417, 'params': {'optimizer': 'adagrad', 'epochs': '6', 'num_layers': '3', 'batch_size': '128', 'hidden_size': '77', 'lr': '0.004802430658713304', 'horizon': '4', 'dropout': '0.3435927279521027', 'input_dim': '4', 'model_name': 'Bi-LSTM', 'seed': '42', 'lag': '49'}}
✅ Dataset regime:
{'temporal_dependence': 'strong', 'stat

# Meta-knoweldge Extraction 


In [9]:
# ==================================================
#  1.Model Architecture Description
# ==================================================
model_description = """
Bidirectional LSTM for multivariate forecasting:
- Input: multivariate sequence
- hidden_size: LSTM units
- num_layers: stacked layers
- dropout applied between layers
"""
# ==================================================
#  1.extract meta-initila trials and meta_knowledge
# ==================================================
with open("C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/meta_data/run_data.json") as f:
    best_trials = json.load(f)
historical_trials = [t["params"] for t in best_trials]

with open("C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/meta_data/meta_features.json") as f:
    meta_features = json.load(f)

raw_features_summary = json.dumps(meta_features["raw_meta_features"], indent=2)
summary_meta_fetaures = json.dumps(meta_features["dataset_regime"], indent=2)
#--------------------------------
 # compute Traget Rmse 
#---------------------------------
target_rmse = meta_builder.compute_target_rmse(best_trials, epsilon=0.08)
#--------------------------------
 # Shots id train data 
#---------------------------------
dataset_sample = json.dumps(clean_train.tail(30).to_dict(orient="records"), indent=2)

# Load Open Source LLM using Ollama



In [5]:
'''
import subprocess
import shutil

def _ollama_installed():
    return shutil.which("ollama") is not None


def _ollama_model_exists(model_name: str):
    try:
        out = subprocess.check_output(["ollama", "list"], text=True)
        return any(model_name in line for line in out.splitlines())
    except Exception:
        return False


def _ollama_pull(model_name: str):
    print(f"⬇️ Pulling Ollama model: {model_name}")
    subprocess.run(["ollama", "pull", model_name], check=True)


def load_llm(
    backend: str = "ollama",
    model_name: str = "qwen2:3b",
    temperature: float = 0.2,
    warn_large: bool = True
):
    backend = backend.lower()

    # --------------------------
    # Ollama (BEST for CPU)
    # --------------------------
    if backend == "ollama":
        if not _ollama_installed():
            raise RuntimeError(
                "❌ Ollama not installed. Install from: https://ollama.com"
            )

        if not _ollama_model_exists(model_name):
            _ollama_pull(model_name)

        if warn_large and any(x in model_name for x in ["13b", "70b"]):
            print("⚠️ WARNING: This model may be too large for CPU.")

        from langchain_community.llms import Ollama
        return Ollama(
            model=model_name,
            temperature=temperature,
            num_ctx=4096,
            num_thread=4   # adjust based on your CPU cores
        )

    # --------------------------
    # HuggingFace (CPU fallback)
    # --------------------------
    elif backend == "hf":
        from langchain_community.llms import HuggingFaceHub

        if warn_large and any(x in model_name.lower() for x in ["13b", "70b"]):
            print("⚠️ WARNING: Large HF model on CPU will be VERY slow.")

        return HuggingFaceHub(
            repo_id=model_name,
            model_kwargs={
                "temperature": temperature,
                "max_new_tokens": 512
            }
        )


    else:
        raise ValueError("Unsupported backend: ollama | openai | hf")'''


#Load LLama
llm = load_llm(
    backend="ollama",
    model_name="llama3:8b",  # medium-small reasoning model
    temperature=0.2
)

# 3. Prompt Template 

In [ ]:
'''
prompt_template = PromptTemplate(
    input_variables=[
        "model_description",
        "num_features",
        "dataset_meta_summary",
        "raw_features_summary",
        "dataset_sample",
        "historical_trials",
        "target_rmse",
        "current_best_rmse"
    ],
    template="""
You are an expert hyperparameter optimization agent specialized in
MULTIVARIATE TIME SERIES FORECASTING using a BIDIRECTIONAL LSTM (BiLSTM).

Your SOLE OBJECTIVE is to MINIMIZE TEST RMSE.
You must suggest new configurations that IMPROVE upon the CURRENT BEST RMSE.
Treat historical_trials as a TABU LIST — NEVER repeat them.

==================================================
OBJECTIVE
==================================================
Current best RMSE: {current_best_rmse}
Target RMSE: < {target_rmse}
Goal: Each suggestion must move the search toward LOWER RMSE.

==================================================
DATASET CONTEXT & META-FEATURES
==================================================
Dataset type: Multivariate time series regression
Number of input variables: {num_features}

Dataset meta-features:
{dataset_meta_summary}

Use meta-features to reason about hyperparameters:
- Seasonality → lag
- Autocorrelation → lag, hidden_size
- Cross-variable interactions → hidden_size, num_layers
- Noise level → dropout, batch_size
- Stationarity & trend → depth, learning rate
- Dataset size → model capacity, epochs

Raw feature summaries (for context only):
{raw_features_summary}

Small data sample (qualitative insight only — do NOT overfit):
{dataset_sample}

==================================================
MODEL CONTEXT
==================================================
Architecture:
- Bidirectional LSTM (capacity is effectively doubled)
- Regression output

Model description:
{model_description}

==================================================
STRICT SEARCH SPACE
==================================================
lag ∈ [12, 16, 24, 32]
hidden_size ∈ [24, 32, 48, 64]
num_layers ∈ [1, 2]
dropout ∈ [0.05, 0.1, 0.15, 0.2]
lr ∈ [0.0005, 0.001, 0.002]
batch_size ∈ [32, 64]
epochs ∈ [20, 30, 40]
optimizer ∈ ["adam", "adamw"]

==================================================
HISTORICAL TRIALS (TABU LIST)
==================================================
{historical_trials}

Rules for this trial:
1. You MUST NOT repeat any configuration from historical_trials.
2. Change AT LEAST TWO hyperparameters from the last trial.
3. At least ONE change MUST be from: dropout, lr, or epochs.
4. Prioritize changes that are most likely to reduce RMSE based on dataset meta-features.
5. Provide concise reasoning for why these changes are expected to improve RMSE.
6. Confirm novelty of configuration before output.

==================================================
OUTPUT FORMAT (STRICT JSON)
==================================================
Output ONLY valid JSON. No markdown, no comments, no extra text.
Return exactly this structure:

{{
  "suggested_params": {{
    "lag": integer,
    "hidden_size": integer,
    "num_layers": integer,
    "dropout": float,
    "lr": float,
    "batch_size": integer,
    "epochs": integer,
    "optimizer": "string"
  }},
  "meta_feature_reasoning": "short explanation linking dataset characteristics to hyperparameter choices",
  "non_repetition_check": "confirmation that this configuration is novel and not in historical_trials",
  "expected_improvement": "short estimate of why RMSE should improve"
}}
No Extrat Data
"""
)'''

In [7]:
llm_phi3mini = load_llm(
    backend="ollama",
    model_name="phi3:mini",
    temperature=0.15
)

⬇️ Pulling Ollama model: phi3:mini


In [8]:
#Define llm chain 
suggest_chain = LLMChain(
    llm=llm_phi3mini,
    prompt=prompt_template
)


In [10]:
name_model = "Bi-LSTM"
experiment_name ="LLM-AutoOpt"
TARGET_COLS =['T (degC)', 'rh (%)', 'p (mbar)', 'wv (m/s)']
clean_train = pd.read_csv('C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/clean_data/clean_train.csv',
    parse_dates=["datetime"],
    index_col="datetime"
    )
clean_test = pd.read_csv('C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/clean_data/clean_test.csv', 
    parse_dates=["datetime"],
    index_col="datetime"
    )

# define ForecastingPipeline
pipeline = ForecastingPipeline(
    clean_train=clean_train,
    clean_test=clean_test,
    target_cols=TARGET_COLS,
    experiment_name=experiment_name
)


In [11]:

#tools
def run_bilstm(params: dict) -> float:
    """
    Runs Bi-LSTM and returns RMSE
    """
    results = pipeline.run("Bi-LSTM", params)
    return float(results["rmse"])



import re
import json
#Flexiable 
def extract_params_and_reasoning(text: str) -> dict:
    """
    Flexible extractor for suggested_params and reasoning from LLM output.
    Works whether JSON is inside ```json``` fences or plain text.
    """
    # ---------------------------
    # 1️⃣ Extract JSON-like substring
    # ---------------------------
    # Try to find a JSON block inside ```json``` first
    json_match = re.search(r"```json(.*?)```", text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1).strip()
    else:
        # Otherwise, find first {...} block in the text
        json_match = re.search(r"\{.*\}", text, re.DOTALL)
        if not json_match:
            raise ValueError("No JSON object found in text")
        json_str = json_match.group(0)

    # ---------------------------
    # 2️⃣ Load JSON safely
    # ---------------------------
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON detected: {e}")

    # ---------------------------
    # 3️⃣ Extract keys
    # ---------------------------
    if "suggested_params" not in data:
        raise KeyError("'suggested_params' not found in JSON")

    reasoning = data.get("meta_feature_reasoning")  # may be None if missing

    return {
        "suggested_params": data["suggested_params"],
        "meta_feature_reasoning": reasoning
    }

# record the trials 
import os
import json
from pathlib import Path

def record_trials(
    params,
    rmse,
    reasoning,
     llm_name,
    base_dir="C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/LLM_trials_memory",  
):
    record = {
        "params": params,
        "rmse": rmse,
        "reasoning": reasoning
    }
    filename = f"trials_history_{llm_name}.json"
    base_dir = Path(base_dir)
    base_dir.mkdir(parents=True, exist_ok=True)   # ensure folder exists

    file_path = base_dir / filename               # real file path

    # Load existing history
    if file_path.exists():
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []   # corrupted/empty file fallback
    else:
        data = []

    # Append new record
    data.append(record)

    # Save back
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

    print(f"✅ Trial recorded → {file_path}")


In [12]:
  # 1️⃣ Ask LLM for new configuration
best_rmse = 1.11

max_trials = 3

for i in range(max_trials):
    print(f"\n===== 🚀 HPO Trial {i+1} =====")

    # 1️⃣ Ask LLM for new configuration
    config_json = suggest_chain.run(
        model_description =  model_description,
        num_features = 4,
        dataset_meta_summary =  dataset_sample ,
        raw_features_summary = raw_features_summary,
        dataset_sample = dataset_sample,
        historical_trials=json.dumps(historical_trials, indent=2),
        current_best_rmse=best_rmse,
        target_rmse=target_rmse
        )

    print("🔧 Suggested JSON:", config_json)
    parsing=  extract_params_and_reasoning(config_json)
    print("Reasoning",parsing["meta_feature_reasoning"])
    # 3️⃣ Run model
    rmse = run_bilstm(parsing["suggested_params"])
    print("📉 RMSE:", rmse)

    # 4️⃣ Update best
    if rmse < best_rmse:
        best_rmse = rmse
        historical_trials.append(parsing["suggested_params"])
        record_trials(parsing["suggested_params"], best_rmse, parsing["suggested_params"])
        print("✅ New best RMSE!")

    # 5️⃣ Early stopping
    if best_rmse <= target_rmse:
        print("🎯 Target RMSE reached!")
        break


===== 🚀 HPO Trial 1 =====


c:\Users\user.IBRAHIM-IK-SZHE\anaconda3\envs\HPO311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


🔧 Suggested JSON: {
  "suggested_params": {
    "lag": 16,
    "hidden_size": 48,
    "num_layers": 2,
    "dropout": 0.15,
    "lr": 0.002,
    "batch_size": 64,
    "epochs": 30,
    "optimizer": "adamw"
  },
  "meta_feature_reasoning": "Increasing the lag to 16 and hidden size to 48 should capture more complex patterns in weather data. A bidirectional LSTM with two layers can better learn from both past and future contexts, which is beneficial for capturing seasonality.",
  "non_repetition_check": true,
    "expected_improvement": "Based on the dataset's longer-term dependencies and non-stationarity in weather patterns, these changes should enhance learning of temporal features leading to a lower RMSE."
}
Reasoning Increasing the lag to 16 and hidden size to 48 should capture more complex patterns in weather data. A bidirectional LSTM with two layers can better learn from both past and future contexts, which is beneficial for capturing seasonality.
Epoch [1/30] Loss: 0.030062 | RMSE

In [15]:
! ollama list

NAME         ID              SIZE      MODIFIED    
llama3:8b    365c0bd3c000    4.7 GB    2 hours ago    


In [11]:
# Load Qwen 3B safely via Ollama
llm = load_llm(
    backend="ollama",
    model_name= "qwen2.5:3b",  # small CPU-friendly model
    temperature=0.15
)


In [12]:

#Define llm chain 
suggest_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

In [3]:
import sys
sys.path.insert(0,r"C:Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/src")
import pandas as pd 
from meta_HPO.meta_llm_tuning import LLMTuning

In [4]:

llm_name ="qwen2.5:3b"
name_model = "Bi-LSTM"
experiment_name ="LLM-AutoOpt"
TARGET_COLS =['T (degC)', 'rh (%)', 'p (mbar)', 'wv (m/s)']
clean_train = pd.read_csv('C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/clean_data/clean_train.csv',
    parse_dates=["datetime"],
    index_col="datetime"
    )
clean_test = pd.read_csv('C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/clean_data/clean_test.csv', 
    parse_dates=["datetime"],
    index_col="datetime"
    )
model_description = """
Bidirectional LSTM for multivariate forecasting:
- Input: multivariate sequence
- hidden_size: LSTM units
- num_layers: stacked layers
- dropout applied between layers
"""
llm_opt =  LLMTuning(name_model,experiment_name,clean_train,clean_test,TARGET_COLS,llm_name,model_description)

c:\Users\user.IBRAHIM-IK-SZHE\Anaconda3\envs\HPO311\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


In [ ]:
meta_trials_data = "C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/meta_data/run_data.json"
meta_features_data = "C:/Users/user.IBRAHIM-IK-SZHE/Meta_LLM_HPO/hyperopt-llm-project/data/meta_data/meta_features.json"
historical_trials,dataset_sample,raw_features_summary,summary_meta_features,target_rmse,best_rmse = llm_opt.extract_metadata(meta_trials_data, meta_features_data, epsilon=0.08)

In [9]:
llm_opt.hpo_optimization(3,historical_trials,dataset_sample,raw_features_summary,target_rmse,best_rmse)


===== HPO Trial 1 =====
Suggested JSON: {
  "suggested_params": {
    "lag": 24,
    "hidden_size": 64,
    "num_layers": 1,
    "dropout": 0.15,
    "lr": 0.001,
    "batch_size": 32,
    "epochs": 30,
    "optimizer": "adam"
  },
  "meta_feature_reasoning": "The dataset has a lag of 24, and reducing dropout to 0.15 from the previous trial's value of 0.3435927279521027 is expected to improve model generalization.",
  "non_repetition_check": "This configuration is novel and not in historical_trials",
  "expected_improvement": "Reducing dropout and increasing hidden_size are likely to reduce overfitting and capture more complex patterns, potentially improving RMSE."
}
Reasoning: The dataset has a lag of 24, and reducing dropout to 0.15 from the previous trial's value of 0.3435927279521027 is expected to improve model generalization.
Epoch [1/30] Loss: 0.026265 | RMSE: 0.162081
Epoch [2/30] Loss: 0.006265 | RMSE: 0.079151
Epoch [3/30] Loss: 0.001961 | RMSE: 0.044266
Epoch [4/30] Loss: 0

KeyboardInterrupt: 